In [18]:
import pandas as pd
import json
from datetime import datetime, timedelta
from sqlalchemy import create_engine
from flask import Flask, jsonify

In [19]:
# read in final data from csv
fire_data = pd.read_csv("fire_data.csv")

In [21]:
# check df dimensions
fire_data.shape

# # checking column types
# for col in fire_data.columns:
#     print(col, type(fire_data[col][0]))

(1376, 27)

In [22]:
# convert table column names to lowercase
fire_data.columns = fire_data.columns.str.lower()
fire_data.shape

(1376, 27)

In [23]:
# some fires are spread through multiple counties, concatenate the county names and types to avoid duplicate entries
fire_data['county'] = fire_data.groupby(['firename'])['county'].transform(lambda x : ', '.join(x))
fire_data['caucus'] = fire_data.groupby(['firename'])['caucus'].transform(lambda x : ', '.join(x))  

# drop duplicate data
fire_data = fire_data.drop_duplicates(subset=['firename'])   
  
# show the dataframe
fire_data.shape

(1023, 27)

### Generating geoJSON

In [7]:
# Custom function to convert DF into geoJSON format
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            if type(row[prop]) != str:
                feature['properties'][prop] = str(row[prop])
            else :
                feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [8]:
# # List properties to be passed to geoJSON
# cols = ['acresburned', 'adminunit', 'archiveyear', 'county', 'extinguished',
#        'fatalities', 'firelocation', 'firename',
#        'searchdescription', 'started', 'structuresdamaged',
#        'structuresdestroyed', 'structuresevacuated', 'structuresthreatened',
#        'timestarted', 'timeextinguished', 'dayofweekstartedname',
#        'dayofweekstartednum', 'duration', 'date_established',
#        'population_jul_2019', 'areasqmi', 'areakm2', 'popdensitypersqmi',
#        'caucus']

In [37]:
fire_geoJSON = df_to_geojson(fire_data , fire_data.drop(['latitude','longitude'], axis=1).columns)

# Write geoJSON formatted text to a text file
with open("fire_data.json", "w") as output:
    json.dump(fire_geoJSON, output)
    
# review geoJSON
len(fire_geoJSON["features"])

1023

### Pushing data to postgreSQL database

In [10]:
# setup connection
rds_connection_string = "postgres:postgres@localhost:5432/ca_fire"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
#  review table names
table_name = engine.table_names()[0]
table_name

'fire_data'

In [12]:
# typecast each datetime column to appropriate date type for SQL import
fire_data.started=pd.to_datetime(fire_data.started)
fire_data.extinguished=pd.to_datetime(fire_data.extinguished)
fire_data.timestarted=pd.to_datetime(fire_data.timestarted)
fire_data.timeextinguished=pd.to_datetime(fire_data.timeextinguished)

# # checking column types
# for col in fire_data.columns:
#     print(col, type(fire_data[col][0]))

In [14]:
fire_data.to_csv("fire_data_1.csv", header=True, index=False)

In [125]:
# write data to postgreSQL database
fire_data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

### Sourcing data from postgreSQL database

In [38]:
# FLask path 1: Pull data for top 10 longest burning fires in each year

query_top_fires_duration = """
SELECT firename, firelocation, archiveyear, started, extinguished, latitude, longitude, 
       acresburned, fatalities, dayofweekstartedname, dayofweekstartednum,
       duration, county, caucus, year_rank 
    FROM (
	SELECT
		*, 
		RANK () OVER ( 
			PARTITION BY p.archiveyear
			ORDER BY duration DESC
		) year_rank 
	FROM
		fire_data p)
AS x WHERE year_rank < 11
"""
# Execute sql query 
data_top_fires_duration = engine.execute(query_top_fires_duration)  

# Pull data table column names
table_headers = engine.execute(query_top_fires_duration)._metadata.keys

# convert to DF
df_top_fires_duration = pd.DataFrame(data_top_fires_duration, columns=table_headers)
# df_top_fires_duration.head()

# Convert refined data frame to geoJSON format
top_fires_duration_geoJSON = df_to_geojson(
    df_top_fires_duration, 
    df_top_fires_duration.drop(['latitude','longitude'], axis=1).columns)
# top_fires_duration_geoJSON

# Write geoJSON formatted text to a text file
with open("top_fires_duration.json", "w") as output:
    json.dump(top_fires_duration_geoJSON, output)

In [39]:
len(top_fires_duration_geoJSON['features'])

70

In [137]:
# FLask path 2: Pull data for top 10 largest (acres affected) fires in each year

query_top_fires_acres = """
SELECT firename, firelocation, archiveyear, started, extinguished, latitude, longitude, 
       acresburned, fatalities, dayofweekstartedname, dayofweekstartednum,
       duration, county, caucus, year_rank 
    FROM (
	SELECT
		*, 
		RANK () OVER ( 
			PARTITION BY p.archiveyear
			ORDER BY acresburned DESC
		) year_rank 
	FROM
		fire_data p)
AS x WHERE year_rank < 11
"""
# Execute sql query 
data_top_fires_acres = engine.execute(query_top_fires_acres)  

# Pull data table column names
table_headers = engine.execute(query_top_fires_acres)._metadata.keys

# convert to DF
df_top_fires_acres = pd.DataFrame(data_top_fires_acres, columns=table_headers)
# df_top_fires_acres.head()

# Convert refined data frame to geoJSON format
top_fires_acres_geoJSON = df_to_geojson(df_top_fires_acres , df_top_fires_acres.drop(['latitude','longitude'], axis=1).columns)
# top_fires_acres_geoJSON

# Write geoJSON formatted text to a text file
with open("top_fires_acres.json", "w") as output:
    json.dump(top_fires_acres_geoJSON, output)

In [138]:
# FLask path 3: Pull data for top 10 deadliest fires overall

query_deadliest = """
SELECT firename, firelocation, archiveyear, started, extinguished, latitude, longitude, 
       acresburned, fatalities, dayofweekstartedname, dayofweekstartednum,
       duration, county, caucus
FROM fire_data WHERE fatalities > 0 ORDER BY fatalities DESC LIMIT 10
"""
# Execute sql query 
data_deadliest_fires = engine.execute(query_deadliest)  

# Pull data table column names
table_headers = engine.execute(query_deadliest)._metadata.keys

# convert to DF
df_deadliest = pd.DataFrame(data_deadliest_fires, columns=table_headers)
# df_deadliest

# Convert refined data frame to geoJSON format
daedliest_fires_geoJSON = df_to_geojson(df_deadliest , df_deadliest.drop(['latitude','longitude'], axis=1).columns)
# daedliest_fires_geoJSON

# Write geoJSON formatted text to a text file
with open("deadliest_fires.json", "w") as output:
    json.dump(daedliest_fires_geoJSON, output)

In [139]:
# FLask path 4: Pull data for all fires
query_all = """SELECT * FROM fire_data;"""
# Execute sql query 
data_all_fires = engine.execute(query_all)  

# Pull data table column names
table_headers = engine.execute(query_all)._metadata.keys

# convert to DF
df_all_fires = pd.DataFrame(data_all_fires, columns=table_headers)
# df_deadliest

# Convert refined data frame to geoJSON format
all_fires_geoJSON = df_to_geojson(df_all_fires , df_all_fires.drop(['latitude','longitude'], axis=1).columns)
# daedliest_fires_geoJSON

# Write geoJSON formatted text to a text file
with open("all_fires.json", "w") as output:
    json.dump(all_fires_geoJSON, output)